# Transformer

## Attention Is All You Need
https://nn.labml.ai/transformers/index.html
1. multi-head attention https://nn.labml.ai/transformers/mha.html
2. encoder and decoder https://nn.labml.ai/transformers/models.htmlch
3. feed forward network https://nn.labml.ai/transformers/feed_forward.html
### 自注意力机制 Scaled Dot-Product Attention
查询（Query）、键（Key）和值（Value）
$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) \times V
$$
a (query) Q: n * d_k (d_k: dimension of key)
a set of (key-value) set: value前的参数取决于key和query的相似度 K: m * d_k; V: m * d_v (d_v: dimension of value)
an (output)： a weighted sum of the values; output: n * d_v
Q与K做内积，内积越大，相似度越高；内积为0，则是正交向量，相似度低
#### 为什么除以sqrt(d_k)？
如果d_k过大，可能导致对应的QK^T过大，导致softmax后的数据过于集中，因此会过早收敛

### multi-head attention
#### 多头注意力机制的引入
时序神经网络不能进行并行计算，因此计算的性能很差
为了解决这些问题，有ConvS2S这种用卷积神经网络代替的方法，但是卷积神经网络的input和output位置是随机的，难以学习一张图上距离较远的两个位置之间的关系
因此引入Transformer的多头注意力机制
#### 多头注意力机制
1.先定义一个head的个数h
2.V、K、Q先各通过一个Linear网络投影到低维（这个Linear网络有参数w可以学习）
d_k = d_v = d_model / h
3.h组低维的V、K、Q进入Scaled Dot-Product Attention
4.将结果concat
5.进入一个Linear网络后输出（这个Linear网络有参数w可以学习）

### encoder and decoder
#### 编码器和解码器的结构
编码器：(x1, x2, ... , xn) -> (z1, z2, ... , zn) 连续生成
解码器：(z1, z2, ... , zn) -> (y1, y2, ... , ym) 一个一个生成，输出vector的长度不同
解码器的auto_regressive：(z1, z2, ... , zn)先生成y1，之后(z1, z2, ... , zn)和y1生成y2，(z1, z2, ... , zn)再和(y1, y2)生成y3，以此类推。因此过去时刻的输出会成为你当前时刻的输入
#### encoder
sub-layer1: multi-head attention
sub-layer2: feed forward
每个sub-layer后增加一个residual层和一个layer-normalization层
LayerNorm(x + SubLayer(x))
每一个层输出的维度d_model = 512（feature的维度）
#### why layer-normalization not batch-normalization?
batch-normalization对每个batch的每个feature做均值和方差
layer-normalization对每个样本做均值和方差
对于数据整体有三个维度，batch_size，seq，feature，因为每个seq的长度可能不同
如果对每个feature求均值和方差，每个feature对应不同的seq的长度不同
#### decoder
sub-layer1: masked multi-head attention 
masked multi-head attention 防止output在i后面的序列影响i的预测
sub-layer2: multi-head attention
sub-layer3: feed forward
每个sub-layer后增加一个residual层和一个layer-normalization层
#### positional encoding
如何让自注意力机制使用seq的时序
input进入模型的第一步是input embedding层，将input的tokens转换为d_model长度的向量
但是这些向量并不具有时序信息
positional encoding层通过一个cos和一个sin函数为每个position编号了
将编号信息与通过了input embedding层的embedding相加，得到的还是d_model长度的向量，但该向量就具备了时序信息

### feed forward network
#### 前馈神经网络
position-wise fully connected feed forward network 其实就是一个简单的MLP 
具体来说就是1个Linear层 + 一层ReLU + 一个Linear层 因此就是一个单隐藏层的MLP

## AN IMAGE IS WORTH 16X16 WORDS: TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE - ViT
https://nn.labml.ai/transformers/vit/index.html